In [1]:
!pip install streamlit darts

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/50/28/fdfe9711d553cfaf217cafc9b8ba21479c663b6504cc96652465b153e04b/streamlit-1.26.0-py2.py3-none-any.whl.metadata
  Using cached streamlit-1.26.0-py2.py3-none-any.whl.metadata (8.0 kB)
  Obtaining dependency information for darts from https://files.pythonhosted.org/packages/0b/29/3d09769040517fe39c2b16c3445a9d88d1290e88ba67a9a08a7071efafce/darts-0.26.0-py3-none-any.whl.metadata
  Using cached darts-0.26.0-py3-none-any.whl.metadata (48 kB)
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/f2/b4/02a0221bd1da91f6e6acdf0525528db24b4b326a670a9048da474dfe0667/altair-5.1.1-py3-none-any.whl.metadata
  Using cached altair-5.1.1-py3-none-any.whl.metadata (8.6 kB)
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a9/c9/c8a7710f2cedcb1db9224fdd4d8307c9

In [ ]:
%%writefile app.py
import requests
import pandas as pd
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.dates as mdates
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import statsmodels.api as sm
import darts
import random
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import StandardScaler
from darts.models import RegressionModel, CatBoostModel, RandomForest, LightGBMModel, XGBModel, RNNModel
from darts.metrics import rmse, mape
from statsmodels.tsa.arima.model import ARIMA
import os

# script_dir = os.path.dirname(__file__)
# config_json = "config.json"
# rel_to_config_json_path = os.path.join(script_dir, config_json)

# with open(rel_to_config_json_path, 'r') as f:
#     # Load JSON data from file
#     json_data = json.load(f)
# path = '/content/drive/MyDrive/Omdena Projects/Weather Prediction for Pakistan/'
# json_data = {
#     "data_columns" : "weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration",
#     "evo_model" : f"{rel_to_config_json_path}evapotranspiration_model.pt",
#     "pre_rate_model" : f"{rel_to_config_json_path}precipitation_rate_model.pt"
# }

# # Load evo_model
# evo_model = RNNModel.load(json_data['evo_model'], map_location='cpu')
# evo_model.to_cpu()

# # Load pre_rate_model
# pre_rate_model = RNNModel.load(json_data['pre_rate_model'], map_location='cpu')
# pre_rate_model.to_cpu()


script_dir = 'C://Users//valer//OneDrive//0_DataScience//99_VSCode//Streamlit'
config_json = "config.json"
rel_to_config_json_path = os.path.join(script_dir, config_json)

with open(rel_to_config_json_path, 'r') as f:
    # Load JSON data from file
    json_data = json.load(f)
path = '/content/drive/MyDrive/Omdena Projects/Weather Prediction for Pakistan/'
json_data = {
    "data_columns" : "weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration",
    "evo_model" : f"{rel_to_config_json_path}evapotranspiration_model.pt",
    "pre_rate_model" : f"{rel_to_config_json_path}precipitation_rate_model.pt"
}

# Load evo_model
evo_model = RNNModel.load('C://Users//valer//OneDrive//0_DataScience//99_VSCode//Streamlit//models//evapotranspiration_model.pt', map_location='cpu')
evo_model.to_cpu()

# Load pre_rate_model
pre_rate_model = RNNModel.load('C://Users//valer//OneDrive//0_DataScience//99_VSCode//Streamlit//models//precipitation_rate_model.pt', map_location='cpu')
pre_rate_model.to_cpu()

data_columns = json_data['data_columns']
now = datetime.now() - relativedelta(days=7)
start = now - relativedelta(months=11)
date_string_end = now.strftime('%Y-%m-%d')
date_string_start = start.strftime('%Y-%m-%d')
date_pred = []
for date in pd.date_range(start=datetime.now() - relativedelta(days=6), periods=10):
    date_pred.append(date.strftime('%Y-%m-%d'))


url = "https://archive-api.open-meteo.com/v1/archive"
cities = [
    { "name": "Karachi", "country": "Pakistan", "latitude": 24.8608, "longitude": 67.0104 }
]
cities_df =[]
for city in cities:
    params = {"latitude":city["latitude"],
            "longitude":city['longitude'],
            "start_date": date_string_start,
            "end_date": date_string_end,
            "daily": data_columns,
            "timezone": "GMT",
            "min": date_string_start,
            "max": date_string_end,
    }
    res = requests.get(url, params=params)
    data = res.json()
    df = pd.DataFrame(data["daily"])
    df["latitude"] = data["latitude"]
    df["longitude"] = data["longitude"]
    df["elevation"] = data["elevation"]
    df["country"] = city["country"]
    df["city"] = city["name"]
    cities_df.append(df)
concat_df = pd.concat(cities_df, ignore_index=True)
concat_df.set_index('time', inplace=True)
print(concat_df.columns)
total_hours = concat_df['precipitation_hours'].sum()
concat_df['precipitation_rate'] = concat_df['precipitation_sum']/total_hours

# generate prediction for max temp
max_temp = concat_df['temperature_2m_max'].values
# Define and fit the model
max_temp_model = ARIMA(max_temp, order=(5,1,0))
max_temp_model = max_temp_model.fit()
# Make predictions
max_temp_predictions = max_temp_model.predict(start=1, end=10)


##generate prediction for evo_transpiration
mean_evo = concat_df['et0_fao_evapotranspiration'].mean()
et0_fao_evapotranspiration = TimeSeries.from_series(concat_df['et0_fao_evapotranspiration'].values, fillna_value=mean_evo)
scaler = StandardScaler()
transformer = Scaler(scaler)
series_transformed = transformer.fit_transform(et0_fao_evapotranspiration)
evo_model.fit(series=series_transformed, verbose=0)
print(f'MODEL: {evo_model}')
evo_preds = evo_model.predict(10, series=series_transformed)
evo_preds = transformer.inverse_transform(evo_preds).univariate_values()
print(evo_preds)
print(len(evo_preds))

print('--------------------')
# ##generate prediction for precipitation rate
#mean_pre = concat_df['et0_fao_evapotranspiration'].mean()
precipitation_rate = TimeSeries.from_series(concat_df['precipitation_rate'].values, fillna_value=0)
scaler = StandardScaler()
transformer = Scaler(scaler)
series_transformed = transformer.fit_transform(precipitation_rate)
pre_rate_model.fit(
    series=series_transformed, verbose=0,
          )
precipitation_rate_preds = pre_rate_model.predict(n=10, series=series_transformed)
precipitation_rate_preds = transformer.inverse_transform(precipitation_rate_preds).univariate_values()
print(precipitation_rate_preds)
print(len(precipitation_rate_preds))

## Visual Element
# headerImage = Image.open('OmdenaHeaderImage.jpg')

## make future value df
predicted_data = {'time': date_pred, 'temperature_2m_max': max_temp_predictions, 'et0_fao_evapotranspiration': evo_preds, 'precipitation_rate': precipitation_rate_preds}
# Create Future value DataFrame
predicted_df = pd.DataFrame(predicted_data)
predicted_df.set_index('time', inplace=True)
print(predicted_df)


##Streamlit app
st.set_page_config(page_title="Weather Prediction", page_icon="⛅")
# st.image(headerImage)
st.title('Karachi Pakistan Weather Prediction')
if concat_df is not None:
    display_df = concat_df[['temperature_2m_max', 'et0_fao_evapotranspiration', 'precipitation_rate']]
    st.write("Current values")
    st.write(display_df.tail(20))

    st.write("Predicted future values")
    st.write(predicted_df)

    # Plot the first column
    st.subheader(f'Plot of temperature_2m_max over time')
    fig1, ax1 = plt.subplots()
    ax1.plot(concat_df.index, concat_df['temperature_2m_max'], label='Historical Value')
    ax1.plot(predicted_df.index, predicted_df['temperature_2m_max'], color='red', label='Predicted Value')
    # Set the locator
    locator = mdates.MonthLocator()  # every month
    fmt = mdates.DateFormatter('%b')

    X = plt.gca().xaxis
    X.set_major_locator(locator)
    X.set_major_formatter(fmt)
    ax1.legend()
    st.pyplot(fig1)

    # Plot the second column
    st.subheader(f'Plot of et0_fao_evapotranspiration over time')
    fig2, ax2 = plt.subplots()
    ax2.plot(concat_df.index, concat_df['et0_fao_evapotranspiration'], label='Historical Value')
    ax2.plot(predicted_df.index, predicted_df['et0_fao_evapotranspiration'], color='red', label='Predicted Value')
    locator = mdates.MonthLocator()  # every month
    fmt = mdates.DateFormatter('%b')

    X = plt.gca().xaxis
    X.set_major_locator(locator)
    X.set_major_formatter(fmt)
    ax2.legend()
    st.pyplot(fig2)

    # Plot the third column
    st.subheader(f'Plot of precipitation_rate over time')
    fig3, ax3 = plt.subplots()
    ax3.plot(concat_df.index, concat_df['precipitation_rate'], label='Historical Value')
    ax3.plot(predicted_df.index, predicted_df['precipitation_rate'], color='red', label='Predicted Value')
    locator = mdates.MonthLocator()  # every month
    fmt = mdates.DateFormatter('%b')

    X = plt.gca().xaxis
    X.set_major_locator(locator)
    X.set_major_formatter(fmt)
    ax3.legend()
    st.pyplot(fig3)



Overwriting app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 86.247.74.158


In [4]:
!streamlit run app.py